In [3]:
from typing import Tuple, Optional
import galois
import numpy as np


def load_text_as_galois_array(path: str, field: Optional[galois.FieldArray] = None, *, force_2d: bool = False):
    """
    Lädt numerische Daten aus einer Textdatei und gibt ein galois-Feld-Array zurück.

    Automatische Formaterkennung:
    - Leerzeichen-/Tab-getrennte Zahlen (Standard von np.loadtxt)
    - CSV (Komma-getrennt)
    - Reine Bitfolge (0/1) ohne Trennzeichen

    Parameter
    - path: Pfad zur Datei (z.B. "ciphertext.txt", "public_key.txt")
    - field: Optional das gewünschte Galois-Feld, z.B. galois.GF(2). Standard: GF(2).
    - force_2d: Erzwingt eine 2D-Form (z.B. für Public Key Matrizen)

    Rückgabe
    - galois-Feld-Array mit den geladenen Werten
    """
    if field is None:
        field = galois.GF(2)

    def _try_load(delimiter):
        return np.loadtxt(path, dtype=int, delimiter=delimiter)

    try:
        # Beliebige Whitespaces als Trennzeichen erlauben
        data = _try_load(None)
    except Exception:
        try:
            # Fallback: CSV
            data = _try_load(",")
        except Exception:
            # Fallback: rohe Ziffernfolge (z.B. 010101...)
            with open(path, "r", encoding="utf-8") as f:
                s = f.read()
            digits = [int(ch) for ch in s if ch in ("0", "1", "2", "3", "4", "5", "6", "7", "8", "9")]
            if len(digits) == 0:
                raise ValueError(f"Konnte keine numerischen Daten aus {path} lesen.")
            data = np.array(digits, dtype=int)

    # Normalisieren der Dimensionen
    if np.isscalar(data):
        data = np.array([data], dtype=int)

    data = np.asarray(data, dtype=int)

    if force_2d and data.ndim == 1:
        data = data.reshape(1, -1)

    return field(data)


def load_ciphertext_and_public_key(
    ciphertext_path: str = "ciphertext.txt",
    public_key_path: str = "public_key.txt",
    field: Optional[galois.FieldArray] = None,
) -> Tuple[galois.FieldArray, galois.FieldArray]:
    """
    Lädt `ciphertext` und `public_key` aus Textdateien als galois-Feld-Arrays.

    - Ciphertext wird als 1D-Vektor normalisiert (falls 2D mit einer Zeile/Spalte).
    - Public Key wird als 2D-Matrix erzwungen; falls als 1xM eingelesen und durch n teilbar, wird auf (k, n) reshaped.
    """
    if field is None:
        field = galois.GF(2)

    ct = load_text_as_galois_array(ciphertext_path, field, force_2d=False)
    n = ct.size

    pk_raw = load_text_as_galois_array(public_key_path, field, force_2d=True)

    pk = pk_raw
    if pk_raw.ndim == 2 and pk_raw.shape[0] == 1:
        total = pk_raw.size
        if total % n == 0:
            k = total // n
            pk_int = np.array(pk_raw, dtype=int).reshape(k, n)
            pk = field(pk_int)

    # Ciphertext nach Möglichkeit zu 1D normalisieren
    if ct.ndim == 2 and 1 in ct.shape:
        ct = ct.reshape(-1)

    return ct, pk

In [4]:
# Laden und Formen prüfen
ciphertext, public_key = load_ciphertext_and_public_key()
print("ciphertext shape:", getattr(ciphertext, "shape", None))
print("public_key shape:", getattr(public_key, "shape", None))

ciphertext shape: (167,)
public_key shape: (57, 167)


In [ ]:
import importlib, attack
importlib.reload(attack)
from attack import InformationSetDecoding

isd = InformationSetDecoding().G_pub(public_key)

decoded = isd.attack(ciphertext)

print("Decoded message:", decoded)

Versuch -989999, Fehlergewicht: 73
Versuch -989991, Fehlergewicht: 87
Versuch -989989, Fehlergewicht: 84
Versuch -989986, Fehlergewicht: 81
Versuch -989984, Fehlergewicht: 79
Versuch -989981, Fehlergewicht: 76
Versuch -989978, Fehlergewicht: 78
Versuch -989975, Fehlergewicht: 79
Versuch -989964, Fehlergewicht: 86
Versuch -989962, Fehlergewicht: 78
Versuch -989959, Fehlergewicht: 71
Versuch -989956, Fehlergewicht: 81
Versuch -989953, Fehlergewicht: 80
Versuch -989951, Fehlergewicht: 87
Versuch -989950, Fehlergewicht: 79
Versuch -989948, Fehlergewicht: 79
Versuch -989945, Fehlergewicht: 76
Versuch -989943, Fehlergewicht: 90
Versuch -989941, Fehlergewicht: 81
Versuch -989937, Fehlergewicht: 86
Versuch -989930, Fehlergewicht: 82
Versuch -989929, Fehlergewicht: 81
Versuch -989925, Fehlergewicht: 91
Versuch -989923, Fehlergewicht: 84
Versuch -989918, Fehlergewicht: 91
Versuch -989917, Fehlergewicht: 78
Versuch -989956, Fehlergewicht: 81
Versuch -989953, Fehlergewicht: 80
Versuch -989951, Feh